## 40. 係り受け解析結果の読み込み（形態素)

In [7]:
class Morph:
    def __init__(self, morph):
        surface, attr = morph.split('\t')
        attr = attr.split(',')
        self.surface = surface
        self.base = attr[6]
        self.pos = attr[0]
        self.pos1 = attr[1]


In [8]:
filename = './ai.ja.txt.parsed'

sentences = []
morphs = []
with open(filename, mode='r') as f:
    for line in f:
        if line[0] == '*':  # 係り受け関係を表す行：スキップ
            continue
        elif line != 'EOS\n':  # 文末以外：Morphを適用し形態素リストに追加
            morphs.append(Morph(line))
        else:  # 文末：形態素リストを文リストに追加
            sentences.append(morphs)
            morphs = []

# 確認
for m in sentences[2]:
    print(vars(m))


{'surface': '人工', 'base': '人工', 'pos': '名詞', 'pos1': '一般'}
{'surface': '知能', 'base': '知能', 'pos': '名詞', 'pos1': '一般'}
{'surface': '（', 'base': '（', 'pos': '記号', 'pos1': '括弧開'}
{'surface': 'じん', 'base': 'じん', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'こうち', 'base': 'こうち', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'のう', 'base': 'のう', 'pos': '助詞', 'pos1': '終助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': 'AI', 'base': '*\n', 'pos': '名詞', 'pos1': '一般'}
{'surface': '〈', 'base': '〈', 'pos': '記号', 'pos1': '括弧開'}
{'surface': 'エーアイ', 'base': '*\n', 'pos': '名詞', 'pos1': '固有名詞'}
{'surface': '〉', 'base': '〉', 'pos': '記号', 'pos1': '括弧閉'}
{'surface': '）', 'base': '）', 'pos': '記号', 'pos1': '括弧閉'}
{'surface': 'と', 'base': 'と', 'pos': '助詞', 'pos1': '格助詞'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '「', 'base': '「', 'pos': '記号', 'pos1': '括弧開'}
{

## 41. 係り受け解析結果の読み込み（文節・係り受け)

In [9]:
class Chunk():
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst = dst
        self.srcs = []


class Sentence():
    def __init__(self, chunks):
        self.chunks = chunks
        for i, chunk in enumerate(self.chunks):
            if chunk.dst != -1:
                self.chunks[chunk.dst].srcs.append(i)


filename = './ai.ja.txt.parsed'

sentences = []
chunks = []
morphs = []
with open(filename, mode='r') as f:
    i = 0
    for line in f:
        if line[0] == '*':  # 係り受け関係を表す行：直前の文節の情報にChunkを適用し文節リストに追加 + 直後の文節の係り先を取得
            if len(morphs) > 0:
                chunks.append(Chunk(morphs, dst))
                morphs = []
            dst = int(line.split(' ')[2].rstrip('D'))
        elif line != 'EOS\n':  # 文末以外：Morphを適用し形態素リストに追加
            morphs.append(Morph(line))
        else:  # 文末：直前の文節の情報にChunkを適用し文節リストに追加 + 文節リストにSentenceを適用し文リストに追加
            if len(morphs) > 0:
                chunks.append(Chunk(morphs, dst))
            if len(chunks) > 0:
                sentences.append(Sentence(chunks))
            morphs = []
            chunks = []
        i += 1

# 確認
for chunk in sentences[1].chunks:
    print([morph.surface for morph in chunk.morphs], chunk.dst, chunk.srcs)


['人工', '知能'] 17 []
['（', 'じん', 'こうち', 'のう', '、', '、'] 17 []
['AI'] 3 []
['〈', 'エーアイ', '〉', '）', 'と', 'は', '、'] 17 [2]
['「', '『', '計算'] 5 []
['（', '）', '』', 'という'] 9 [4]
['概念', 'と'] 9 []
['『', 'コンピュータ'] 8 []
['（', '）', '』', 'という'] 9 [7]
['道具', 'を'] 10 [5, 6, 8]
['用い', 'て'] 12 [9]
['『', '知能', '』', 'を'] 12 []
['研究', 'する'] 13 [10, 11]
['計算', '機', '科学'] 14 [12]
['（', '）', 'の'] 15 [13]
['一', '分野', '」', 'を'] 16 [14]
['指す'] 17 [15]
['語', '。'] 34 [0, 1, 3, 16]
['「', '言語', 'の'] 20 []
['理解', 'や'] 20 []
['推論', '、'] 21 [18, 19]
['問題', '解決', 'など', 'の'] 22 [20]
['知的', '行動', 'を'] 24 [21]
['人間', 'に'] 24 []
['代わっ', 'て'] 26 [22, 23]
['コンピューター', 'に'] 26 []
['行わ', 'せる'] 27 [24, 25]
['技術', '」', '、', 'または', '、'] 34 [26]
['「', '計算', '機'] 29 []
['（', 'コンピュータ', '）', 'による'] 31 [28]
['知的', 'な'] 31 []
['情報処理', 'システム', 'の'] 33 [29, 30]
['設計', 'や'] 33 []
['実現', 'に関する'] 34 [31, 32]
['研究', '分野', '」', 'と', 'も'] 35 [17, 27, 33]
['さ', 'れる', '。'] -1 [34]


## 42. 係り元と係り先の文節の表示

In [10]:
sentence = sentences[1]

for chunk in sentence.chunks:
    if int(chunk.dst) != -1:
        passive = "".join(
            [m.surface if m.pos != '記号' else '' for m in chunk.morphs])
        active = "".join(
            [m.surface if m.pos != '記号' else '' for m in sentence.chunks[chunk.dst].morphs])
        print(passive, active, sep="    ")


人工知能    語
じんこうちのう    語
AI    エーアイとは
エーアイとは    語
計算    という
という    道具を
概念と    道具を
コンピュータ    という
という    道具を
道具を    用いて
用いて    研究する
知能を    研究する
研究する    計算機科学
計算機科学    の
の    一分野を
一分野を    指す
指す    語
語    研究分野とも
言語の    推論
理解や    推論
推論    問題解決などの
問題解決などの    知的行動を
知的行動を    代わって
人間に    代わって
代わって    行わせる
コンピューターに    行わせる
行わせる    技術または
技術または    研究分野とも
計算機    コンピュータによる
コンピュータによる    情報処理システムの
知的な    情報処理システムの
情報処理システムの    実現に関する
設計や    実現に関する
実現に関する    研究分野とも
研究分野とも    される


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

In [11]:
sentence = sentences[1]

for chunk in sentence.chunks:
    if int(chunk.dst) != -1:
        if ("名詞" in [m.pos for m in chunk.morphs]) and ("動詞" in [m.pos for m in sentence.chunks[chunk.dst].morphs]):
            passive = "".join(
                [m.surface if m.pos != '記号' else '' for m in chunk.morphs])
            active = "".join(
                [m.surface if m.pos != '記号' else '' for m in sentence.chunks[chunk.dst].morphs])
            print(passive, active, sep="    ")

道具を    用いて
知能を    研究する
一分野を    指す
知的行動を    代わって
人間に    代わって
コンピューターに    行わせる
研究分野とも    される


## 44. 係り受け木の可視化

In [12]:
# from graphviz import Digraph
# dot = Digraph(comment='The Round Table')
# dot  #doctest: +ELLIPSIS
# dot.node('A', 'King Arthur')
# dot.node('B', 'Sir Bedevere the Wise')
# dot.node('L', 'Sir Lancelot the Brave')
# dot.edges(['AB', 'AL'])
# dot.edge('B', 'L', constraint='false')
# print(dot.source)  # doctest: +NORMALIZE_WHITESPACE
# dot.render('test-output/round-table.gv', view=True)

## 45. 動詞の格パターンの抽出

In [13]:
with open('./ans45.txt', 'w') as f:
    for sentence in sentences:
        for chunk in sentence.chunks:
            for morph in chunk.morphs:
                if morph.pos == '動詞':  # chunkの左から順番に動詞を探す
                    cases = []
                    for src in chunk.srcs:  # 見つけた動詞の係り元chunkから助詞を探す
                        cases = cases + \
                            [morph.surface for morph in sentence.chunks[src].morphs if morph.pos == '助詞']
                    if len(cases) > 0:  # 助詞が見つかった場合は重複除去後辞書順にソートして出力
                        cases = sorted(list(set(cases)))
                        line = '{}\t{}'.format(morph.base, ' '.join(cases))
                        print(line, file=f)
                    break


## 46. 動詞の格フレーム情報の抽出

In [17]:
with open('./ans46.txt', 'w') as f:
    for sentence in sentences:
        for chunk in sentence.chunks:
            for morph in chunk.morphs:
                if morph.pos == '動詞':  # chunkの左から順番に動詞を探す
                    cases = []
                    cases_concrete = []
                    for src in chunk.srcs:  # 見つけた動詞の係り元chunkから助詞を探す
                        case = [
                            morph.surface for morph in sentence.chunks[src].morphs if morph.pos == '助詞']
                        if len(case) > 0:  # 助詞を含むchunkの場合は助詞と項を取得
                            cases = cases + case
                            cases_concrete.append(''.join(
                                morph.surface for morph in sentence.chunks[src].morphs if morph.pos != '記号'))
                    if len(cases) > 0:  # 助詞が1つ以上見つかった場合は重複除去後辞書順にソートし、項と合わせて出力
                        cases = sorted(list(set(cases)))
                        line = '{}\t{}\t{}'.format(
                            morph.base, ' '.join(cases), ' '.join(cases_concrete))
                        print(line, file=f)
                    break

## 47. 機能動詞構文のマイニング

In [19]:
with open('./ans47.txt', 'w') as f:
    for sentence in sentences:
        for chunk in sentence.chunks:
            for morph in chunk.morphs:
                if morph.pos == '動詞':  # chunkの左から順番に動詞を探す
                    # 見つけた動詞の係り元chunkが「サ変接続名詞+を」で構成されるか確認
                    for i, src in enumerate(chunk.srcs):
                        if len(sentence.chunks[src].morphs) >= 2:
                            if sentence.chunks[src].morphs[-2].pos1 == 'サ変接続' and sentence.chunks[src].morphs[-1].surface == 'を':
                                predicate = ''.join(
                                    [sentence.chunks[src].morphs[-2].surface, sentence.chunks[src].morphs[-1].surface, morph.base])
                                cases = []
                                modi_chunks = []
                                # 残りの係り元chunkから助詞を探す
                                for src_r in chunk.srcs[:i] + chunk.srcs[i + 1:]:
                                    case = [
                                        morph.surface for morph in sentence.chunks[src_r].morphs if morph.pos == '助詞']
                                    if len(case) > 0:  # 助詞を含むchunkの場合は助詞と項を取得
                                        cases = cases + case
                                        modi_chunks.append(''.join(
                                            morph.surface for morph in sentence.chunks[src_r].morphs if morph.pos != '記号'))
                                if len(cases) > 0:  # 助詞が1つ以上見つかった場合は重複除去後辞書順にソートし、項と合わせて出力
                                    cases = sorted(list(set(cases)))
                                    line = '{}\t{}\t{}'.format(
                                        predicate, ' '.join(cases), ' '.join(modi_chunks))
                                    print(line, file=f)
                                break

## 48. 名詞から根へのパスの抽出

In [21]:
sentence = sentences[1]
for chunk in sentence.chunks:
    if "名詞" in [m.pos for m in chunk.morphs]:  # chunkに名詞が含まれているとき
        path = [
            ''.join(morph.surface for morph in chunk.morphs if morph.pos != '記号')]
        target_chunk = chunk
        while target_chunk.dst != -1:  #dstがなくなるまで
            path.append(''.join(
                [morph.surface for morph in sentence.chunks[target_chunk.dst].morphs if morph.pos != '記号']))
            target_chunk = sentence.chunks[target_chunk.dst]
        print(' -> '.join(path))


人工知能 -> 語 -> 研究分野とも -> される
じんこうちのう -> 語 -> 研究分野とも -> される
AI -> エーアイとは -> 語 -> 研究分野とも -> される
エーアイとは -> 語 -> 研究分野とも -> される
計算 -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
概念と -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
コンピュータ -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
知能を -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
一分野を -> 指す -> 語 -> 研究分野とも -> される
語 -> 研究分野とも -> される
言語の -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
理解や -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
人間に -> 代わって -> 行わせる -> 技術または -> 研究

## 49. 名詞間の係り受けパスの抽出

In [24]:
from itertools import combinations
import re

sentence = sentences[1]
nouns = []
for i, chunk in enumerate(sentence.chunks):
    if '名詞' in [morph.pos for morph in chunk.morphs]:  # 名詞を含む文節を抽出
        nouns.append(i)
for i, j in combinations(nouns, 2):  # 名詞を含む文節のペアごとにパスを作成
    path_i = []
    path_j = []
    while i != j:
        if i < j:
            path_i.append(i)
            i = sentence.chunks[i].dst
        else:
            path_j.append(j)
            j = sentence.chunks[j].dst
    if len(path_j) == 0:  # 1つ目のケース
        chunk_X = ''.join([morph.surface if morph.pos !=
                           '名詞' else 'X' for morph in sentence.chunks[path_i[0]].morphs])
        chunk_Y = ''.join([morph.surface if morph.pos !=
                          '名詞' else 'Y' for morph in sentence.chunks[i].morphs])
        chunk_X = re.sub('X+', 'X', chunk_X)
        chunk_Y = re.sub('Y+', 'Y', chunk_Y)
        path_XtoY = [chunk_X] + [''.join(morph.surface for morph in sentence.chunks[n].morphs)
                                 for n in path_i[1:]] + [chunk_Y]
        print(' -> '.join(path_XtoY))
    else:  # 2つ目のケース
        chunk_X = ''.join([morph.surface if morph.pos !=
                          '名詞' else 'X' for morph in sentence.chunks[path_i[0]].morphs])
        chunk_Y = ''.join([morph.surface if morph.pos !=
                          '名詞' else 'Y' for morph in sentence.chunks[path_j[0]].morphs])
        chunk_k = ''.join(
            [morph.surface for morph in sentence.chunks[i].morphs])
        chunk_X = re.sub('X+', 'X', chunk_X)
        chunk_Y = re.sub('Y+', 'Y', chunk_Y)
        path_X = [
            chunk_X] + [''.join(morph.surface for morph in sentence.chunks[n].morphs) for n in path_i[1:]]
        path_Y = [
            chunk_Y] + [''.join(morph.surface for morph in sentence.chunks[n].morphs) for n in path_j[1:]]
        print(' | '.join([' -> '.join(path_X), ' -> '.join(path_Y), chunk_k]))


XX | （YYのう、、 | 語。
XX | Y -> 〈エーアイ〉）とは、 | 語。
XX | 〈Y〉）とは、 | 語。
XX | 「『Y -> （）』という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す | 語。
XX | Yと -> 道具を -> 用いて -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す | 語。
XX | 『Y -> （）』という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す | 語。
XX | Yを -> 用いて -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す | 語。
XX | 『Y』を -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す | 語。
XX | Yする -> 計算機科学 -> （）の -> 一分野」を -> 指す | 語。
XX | YYY -> （）の -> 一分野」を -> 指す | 語。
XX | YY」を -> 指す | 語。
XX -> Y。
XX -> 語。 | 「Yの -> 推論、 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術」、または、 | 研究分野」とも
XX -> 語。 | Yや -> 推論、 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術」、または、 | 研究分野」とも
XX -> 語。 | Y、 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術」、または、 | 研究分野」とも
XX -> 語。 | YYなどの -> 知的行動を -> 代わって -> 行わせる -> 技術」、または、 | 研究分野」とも
XX -> 語。 | YYを -> 代わって -> 行わせる -> 技術」、または、 | 研究分野」とも
XX -> 語。 | Yに -> 代わって -> 行わせる -> 技術」、または、 | 研究分野」とも
XX -> 語。 | Yに -> 行わせる -> 技術」、または、 | 研究分野」とも
XX -> 語。 | Y」、または、 | 研究分野」とも
XX -> 語。 | 「YY -> （コンピ